# Review 5- Programming Styles: CPS and Lazy Evaluations

# PART (1/2) :: Continuation Passing Style (CPS)

### -What is CPS?
   Normally a function takes some input then returns some value and then the caller of the function continues to do something with that returned value. 

   In CPS, instead of returning some value, the function would take a continuation function, which represents what the caller would do with the returned value, as a first-class parameter, and calls the continuation where the normal function returns the result. CPS is a style of programming where every function will have some extra argument called the 'continuation'. 
   
A continuation is a function that is passed in and specifies what the caller
wishes to do with the result that has been computed. 

### -What's the big deal? I don't like CPS.
Ultimately, the benefit of using CPS is computational resource management. If you are not limited by memory or processor resources, there's probably not a reason to use CPS in your programming style.

The benefit of CPS is that the style aims to free up computational resources by removing processor/memory allocation that is no londer necessary. This can be done by storing only the latest argument values or evaluations from a recursive function so that it never goes into the more than 1-deep when implimenting its own recursion. 

When not using CPS in your programming style, the function is not limited by how much resources it uses to run. Possibly going many thousands of times into a recursion loop. Storing each evaluation, instantiation of methods and variables, etc. each and every time the function is called in the recursion loop until it is finished. As you might imagine, this can get out of control...

### -What is a continuation in CPS?
A continuation is a function that is passed in and specifies what the caller
wishes to do with the result that has been computed.

Take for instance, a function `func` that takes in an integer and returns an integer.
~~~
def func(x: Int): Int = {
     // .. do some work to compute result .. 
     return result
}
~~~

In the CPS, this function is now written as

~~~
def func-k(x: Int, k: Int => Int) : Int = {
    //  .. do some work to compute result ..
    k(result) // Pass the result onto the continuation.
}
~~~

Note that `func-k` takes in an extra argument `k` called continuation. It
is the function through which the caller specifies what they want done with
the result of the call. Rather than return the result and make the caller operate
on it, the caller simply bundles up the results and passes it all in.

Let us look at a concrete example. First take a look at these three functions defined below.

### -What can I put inside a continuation?
1) Technically, anything you want. But usually just function OUTPUTS that are used as INPUTS for the next evaluation to limit the depth of a recursive loop.

2) However, you can also pass ERRORS instead of letting the compiler raise an exception. This lets you deal with the error at the very end (assuming it gets there).



In [2]:
// This function takes an integer x and returns x + 1
def addOne(x: Int): Int = {
    val result = x + 1
    result
}

defined function addOne

In [3]:
addOne(5)

res2: Int = 6

#### Arguments: 
#### x : Standard argument value.

#### k : Continuation. Passes the results of the previous call to the function onto the next function call.

#### Types:
#### x : Integer
#### k : A delta function (=>) that takes an integer (the previous x) and outputs a new integer (the previous function output). 

### Structurally, continuations in general usually look like this k argument.

In [4]:
// In CPS, we would write...
def addOne_cps(x: Int, k: Int => Int): Int = {
    val result = x + 1
    k(result)
}

defined function addOne_cps

In [5]:
val square: Int => Int = y => y * y
addOne_cps(5, square)

square: Int => Int = ammonite.$sess.cmd4$Helper$$Lambda$1855/750474367@142da80e
res4_1: Int = 36

In [6]:
// Use generics to allow any return type for k
def addTwo_cps[T](x: Int, k: Int => T): T = {
    val result = x + 2
    k(result)
}

defined function addTwo_cps

In [7]:
addTwo_cps(5, square)

res6: Int = 49

In [8]:
addTwo_cps(5, x => List(x))

res7: List[Int] = List(7)

### Another Example
Here, we will write a function `madd_k` which will-
- call `multiply_k` on x, y and pass a continuation `k1` to `multiply_k`
- The continuation `k1` should-
  1. Call addUp_k
  2. Pass the result on to continuation k

In [11]:
// (A) => (B) => (C)

def addUp_k(n: Int, m: Int, B: Int => Int): Int = {
    B(n + m)
}

def multiply_k(x: Int, y: Int, A: Int => Int): Int = {
    A( x * y)
}

def madd_k(x: Int, y: Int, C: Int => Int): Int ={
    multiply_k(x,y, v1=> addUp_k(v1, 4, C) )
    //def B_output(v1: Int): Int = addUp_k(v1, 4, C)
   // multiply_k(x, y, B_output)
}

defined function addUp_k
defined function multiply_k
defined function madd_k

In [12]:
madd_k(2, 3, input => input * 2)

res11: Int = 20

#### [FIRST]
##### -Run multiply_k (x*y) to get 2x3=6 and pass that result to its continuation : (A).

#### [SECOND]
##### - Take the continuation from multiply_k (A=6) and give it to add_k to get 6 + 3 = 9 and pass that result to its continuation : (B).

#### [THIRD]
##### -Take the continuation from add_k (B=9) and evaluate it on the continuation from madd_k : (C).

#### ORDER OF OPERATIONS:
#### C [     B [     A [x * y]     + 3]        * 2]      =      [     [     [2 * 3]     + 3]     * 2] 

### Example: Side Effects
Given the following functions, change them to use continuations:

In [13]:
// Function with side effects
def printFive(f: Int => String): Int = {
    val five = 5
    println("My value is: "+ f(five))
    five
}

defined function printFive

In [14]:
def printFive_cps(f: Int => String, k: Int => Int): Int = {
    // Your Code
    val five = 5
    print("My value now is: " + f(five))
    k(five)
}

defined function printFive_cps

In [15]:
printFive(x => x.toString)

My value is: 5


res14: Int = 5

In [16]:
printFive_cps(x => x.toString, y => y * 10)

My value now is: 5

res15: Int = 50

# Error Handling

So far we have worked with continuation without any error. Now let's see how to handle error cases within continuation which is called the "error continuation". It is called whenever the program encounters an error.

The type of our CPS function will become:

fun_k(arg: ..., k: ResultType=> T, err_k: Unit => T ) : T

Here, if some error arises in the computation that would normally be handled by throwing an exception, we will call the error continuation instead.



In [17]:
def error_continuation_ex[T](x: Int,  k: Int => T, err_k: () => T): T = {
    x match {
        case 1 => k(1)
        case x if x > 1 => error_continuation_ex(x - 1, k, err_k)
        case _ => err_k()
    }
}

defined function error_continuation_ex

In [18]:
println(error_continuation_ex(1, x => "I've found the one", () => "Havn't found the one yet!"))
println(error_continuation_ex(10, x => "It took me a while! But I've found the one", () => "Havn't found the one yet!"))
println(error_continuation_ex(0, x => "I've found the one", () => "Havn't found the one yet!"))

I've found the one
It took me a while! But I've found the one
Havn't found the one yet!


### Exercise: Fibonacci

In [19]:
def fibonacci(n: Int): Int = {
    if (n < 2){
        1
    } else {
        fibonacci(n-1) + fibonacci(n-2)
    }    
}

// Fibonacci in Contiuation Passing Style
def fib_cps (n: Int, k: (Int => Int)) : Int = n match{
    case 0 => k(0)
    case 1 => k(1)
    case _ => {
        fib_cps(n-1, v1=> 
                      fib_cps(n-2, 
                              v2=> k(v1+v2)))
    }
    
}
    
    
    
    
    
//     case 0 => k(0)
//     case 1 => k(1)
//     case _ => fib_cps(n-1, (a: Int) => // This is a continuation for adding the first number
//         fib_cps(n-2, (b: Int) => // This is another continuation for adding the second number
//             k(a+b))) // This is the final continuation for adding the those two numbers
// }

defined function fibonacci
defined function fib_cps

In [20]:
assert(fib_cps(0, (x: Int) => x) == 0)
assert(fib_cps(6, (x: Int) => x) == 8)
assert(fib_cps(8, (x: Int) => x) == 21)

# Key takeaway

- We add an extra continuation argument to every function call in the program.
- We transform the program so that all function calls happen at the tail position.
- Finally, we __hope__ that the compiler/interpreter in all its goodness will optimize the tail call away.

# Part (2/2) :: Lazy Evaluations

Lazy Evaluation is another evaluation strategy. This method delays the evaluation of an expression until its value is needed. Lazy evaluation can provide few benefits when compared to normal strict evaluation. They are as follows: 

1. It can provide performance enhancement by not doing calculations until needed — and they may not be done at all if the calculation is not used.

2. It can increase the response time of applications by postponing the heavy operations until required.

Scala supports lazy evaluation using two approaches:

1. Call by NEED
2. "lazy" keyword

### Lazy Eval Scala Approach #1:  Call by NEED

In Call-by-Name we just prepend the => symbol in the argument type. The Call-by-Name functions evaluate the passed-in expression’s value for every single use.

In [21]:
// Call by Name
def provideNumber(): Int = {
    println("I'm searching for a number to give..")
    10 // always returns 10
}

def provideNumber_different(): Int = {
    println("I'm searching for a....different....number to give..")
    50 // always returns 10
}

def callByNameFunc(n: => Int): Unit = {
    val result = n + n + n + 5
    print(s"Result is : ${result}")
}

defined function provideNumber
defined function provideNumber_different
defined function callByNameFunc

**Question:** How many times will the string in provideNumber() be called??

**Question:** How many times will the string in callByNameFunc() be called??

**Answer**: ???

In [210]:
println("\n\nUsing -- provideNumber -- for implimenting the delta function in callByNameFunc::")
callByNameFunc(provideNumber)



Using -- provideNumber -- for implimenting the delta function in callByNameFunc::
I'm searching for a number to give..
I'm searching for a number to give..
I'm searching for a number to give..
Result is : 35

In [211]:
println("\n\nUsing -- provideNumber_different -- for implimenting the delta function in callByNameFunc::")
callByNameFunc(provideNumber_different)



Using -- provideNumber_different -- for implimenting the delta function in callByNameFunc::
I'm searching for a....different....number to give..
I'm searching for a....different....number to give..
I'm searching for a....different....number to give..
Result is : 155

### Lazy Eval Scala Approach #2:  Just put the word lazy before a val decleration

The compiler does not immediately evaluate the bound expression of a lazy val. It evaluates the variable only on its first access. Upon initial access, the compiler evaluates the expression and stores the result in the lazy val. Whenever we access this val at a later stage, no execution happens, and the compiler returns the result.

To designate a val as lazy, we simply need to add the lazy keyword in front of the variable declaration.

In [212]:
lazy val myNumber: Int = {
    println("I'm assigning number 7 to myNumber")
    7
}

myNumber: Int = <lazy>


In [213]:
val result: Int = myNumber + 20

I'm assigning number 7 to myNumber


result: Int = 27


In [214]:
myNumber

res105: Int = 7


In [215]:
// Exercise: Re-write callByNameFunc such that provideNumber is called only once.
// Hint: Use lazy keyword to accomplish the task

def callByNameFuncRevised(n: => Int): Unit = {
    // BEGIN Solution
    lazy val n1 = n
    val result = n1 + n1 + n1 + 5 
    // End Solution
    print(s"Result is : ${result}")
}

callByNameFuncRevised: (n: => Int)Unit


In [216]:
callByNameFuncRevised(provideNumber())

I'm searching for a number to give..
Result is : 35

### Hybrid Lazy Evaluations: Using both the 'lazy keyword' and 'call when needed'.

In [217]:
println("Run their functions only when this value is observed:")
lazy val Lazy_Run_Functions: Unit = {callByNameFunc(provideNumber)}

Run their functions only when this value is observed:


Lazy_Run_Functions: Unit = <lazy>


In [218]:
Lazy_Run_Functions

I'm searching for a number to give..
I'm searching for a number to give..
I'm searching for a number to give..
Result is : 35

### Lazy Functors - filter vs. withFilter - Methods in the List API

In [219]:
filter: Runs the input function on ALL members of the list IMMEDIATELY.

withFilter: Only runs the input function

<console>: 4: error: ';' expected but ':' found.

In [220]:
def isLessThan30(x: Int): Boolean = {
    println(s"Is ${x} less than 30?")
    println(x < 30)
    x < 30
}

val myList: List[Int] = List(12, 78, 23, 56, 45, 29, 45, 9)

isLessThan30: (x: Int)Boolean
myList: List[Int] = List(12, 78, 23, 56, 45, 29, 45, 9)


In [221]:
// Normal Filter operation
val newList = myList.filter(isLessThan30) // Filter creates a new collection
println(newList)

Is 12 less than 30?
true
Is 78 less than 30?
false
Is 23 less than 30?
true
Is 56 less than 30?
false
Is 45 less than 30?
false
Is 29 less than 30?
true
Is 45 less than 30?
false
Is 9 less than 30?
true
List(12, 23, 29, 9)


newList: List[Int] = List(12, 23, 29, 9)


In [222]:
// withFilter operation 
val newListLazy = myList.withFilter(isLessThan30) // Does not evaluate immediately, just returns the iterable but not
// a new collection


newListLazy: scala.collection.generic.FilterMonadic[Int,List[Int]] = scala.collection.TraversableLike$WithFilter@21504647


In [223]:
println(newListLazy)

scala.collection.TraversableLike$WithFilter@21504647


In [224]:
newListLazy.foreach(n => n) // Does not create a new list

Is 12 less than 30?
true
Is 78 less than 30?
false
Is 23 less than 30?
true
Is 56 less than 30?
false
Is 45 less than 30?
false
Is 29 less than 30?
true
Is 45 less than 30?
false
Is 9 less than 30?
true


#### So does newListLazy now know the booleans evaluated on each member of the list??

In [182]:
println(newListLazy)

scala.collection.TraversableLike$WithFilter@2436e955


### Advanced Lazy programming: Streams

Streams are similar to list in scala. The only difference is that, in scala stream value will only be calculated when needed. Hence increases the performance of the program by not loading the value at once.

Streams in Scala can be declared as below:

#### Stream Exercise

Use methods native to the List API inside of a Stream

In [446]:
// Convert a Range object of values to a Stream object
val stream = (0 to 100).toStream
// BEFORE Evaluating any index location of the Stream list, it has ZERO knowledge on its contents.
stream

stream: scala.collection.immutable.Stream[Int] = Stream(0, ?)
res280: scala.collection.immutable.Stream[Int] = Stream(0, ?)


In [447]:
// Evaluate at a position that you want and it will know ALL PRECEEDING values up to that point
stream(10)
stream

res281: scala.collection.immutable.Stream[Int] = Stream(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ?)


In [448]:
stream(30)
stream

res282: scala.collection.immutable.Stream[Int] = Stream(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, ?)


In [435]:
// Pushing the stream object to a new value 
// does NOT bring with it all the information that stream list learned up to that point
val stream_map = stream.map {_ * 10}
stream_map

stream_map: scala.collection.immutable.Stream[Int] = Stream(0, ?)
res270: scala.collection.immutable.Stream[Int] = Stream(0, ?)


In [438]:
stream_map(20)
stream_map

res273: scala.collection.immutable.Stream[Int] = Stream(0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, ?)


# REVIEW SUMMARY: CPS and Lazy Programming Styles

## Style 1: CPS

#### -Passes information in a dummy variable called the 'continuation' on the input needed for subsequent evaluations of a function.

#### -Allows the programmer to limit the depth of a recursion loop to a depth of one. 

#### -Useful for resource limited programming.

## Style 2: Lazy

#### -Does not evaluate the functions or values used until it is needed (when it is observed).

#### -Before the value is observed, Scala is totally unaware of what values or functions are within it.

#### -Allows the programmer to hold off on implimenting a series of functions until they have the necessary information.

#### -The Stream object is similar to a List (has all the same methods) but it will not evaluate the members of the Stream list until they are called. This is a list structured form of Lazy programming.

## Good Luck!!